# Train Driving Model

In [1]:
import os
import csv

import cv2
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
samples = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    # Skip header line
    header = next(reader)
    for line in reader:
        samples.append(line)

In [3]:
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [4]:
def generator(samples, batch_size=32):
    num_samples = len(samples) - (len(samples) % batch_size)
    shuffle(samples)
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            #yield X_train, y_train
            yield sklearn.utils.shuffle(X_train, y_train)

In [5]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [6]:
from keras.models import Sequential
from keras.layers import Cropping2D, Dense, Flatten, Lambda

Using TensorFlow backend.


In [7]:
ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=(160, 320, 3)))
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

In [8]:
model.compile(loss='mse', optimizer='adam')
history = model.fit_generator(
    train_generator,
    samples_per_epoch= len(train_samples),
    validation_data=validation_generator, 
    nb_val_samples=len(validation_samples),
    nb_epoch=3,
) 

Epoch 1/3
6400/6428 [============================>.] - ETA: 0s - loss: 2.1682

/usr/local/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6432/6428 [==============================] - 22s - loss: 2.1678 - val_loss: 3.0125
Epoch 2/3
6432/6428 [==============================] - 21s - loss: 5.7667 - val_loss: 3.5353
Epoch 3/3
6432/6428 [==============================] - 20s - loss: 7.7810 - val_loss: 1.4793


In [9]:
model.save('./data/model.h5')

In [10]:
history.history

{'loss': [2.1678077522201917, 5.7666869566808296, 7.7810129745682675],
 'val_loss': [3.0125189388499543, 3.5352720139073393, 1.4792906525088292]}